In [ ]:
pip install pexels_api python-dotenv

In [ ]:
import os
import csv


from pexels_api import API
from dotenv import load_dotenv

load_dotenv()

# set up API
api = API("NML9LYjorpAVsQdHb6g8eRMzwNajuoTl0QxwdB85aWN2OkE6iBYMmCRu")

# set up search criteria
search_terms = ['lemon', 'lychee', 'grape', 'kiwi', 'apple'] # keywords for search
num_images = 150 # number of images to download for each keyword
page_size = 80 # maximum number of images per page

# loop through each search term
for term in search_terms:
    # create folder to save images and CSV file
    folder_name = f'{term}_images'
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    csv_file = os.path.join(folder_name, f'{term}_urls.csv')
    
    # search for images based on term
    photos = []
    for page in range(1, (num_images // page_size) + 2):
        results = api.search(term, page=page, results_per_page=page_size)['photos']
        if not results:
            break
        photos.extend(results)
        if len(photos) >= num_images:
            break
    photos = photos[:num_images]

    # save URLs to CSV file
    with open(csv_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['image_id', 'image_url'])
        for i, photo in enumerate(photos):
            image_id = f'{term}_{i}'
            image_url = photo['src']['original']
            writer.writerow([image_id, image_url])
            print(f'Saved URL for {image_id} to {csv_file} file.')

In [ ]:
import os
import csv
import requests

# set up search criteria
search_terms = ['lemon', 'lychee', 'grape', 'kiwi', 'apple'] # keywords for search

# loop through each search term
for term in search_terms:
    # read CSV file to get image URLs and image IDs
    csv_file = os.path.join(f'{term}_images', f'{term}_urls.csv')
    with open(csv_file, 'r') as f:
        reader = csv.reader(f)
        next(reader) # skip header row
        for row in reader:
            image_id = row[0]
            image_url = row[1]
            response = requests.get(image_url)
            file_name = os.path.basename(image_url)
            file_ext = os.path.splitext(file_name)[1]
            file_name = f'{image_id}{file_ext}'
            file_path = os.path.join(f'{term}_images', file_name)
            with open(file_path, 'wb') as f:
                f.write(response.content)
                print(f'Saved {file_name} to {term}_images folder.')


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# set up search criteria
search_terms = ['lemon', 'lychee', 'grape', 'kiwi', 'apple'] # keywords for search

# set up parameters
img_size = (128, 128) # size to resize images
num_classes = len(search_terms) # number of fruit classes

# load images and labels
images = []
labels = []
for i, term in enumerate(search_terms):
    folder_name = f'{term}_images'
    for filename in os.listdir(folder_name):
        image_path = os.path.join(folder_name, filename)
        try:
            # read image and resize
            img = cv2.imread(image_path)
            img = cv2.resize(img, img_size)
            images.append(img)
            labels.append(i)
        except Exception as e:
            print(f'Error loading {image_path}: {e}')

# convert to numpy arrays
images = np.array(images)
labels = np.array(labels)

# split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# convert labels to categorical
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# create model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# train model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
